In [3]:
# ruff: noqa: T201

import numpy as np
import gemmi
import reciprocalspaceship as rs
from matplotlib import pyplot as plt

from meteor.diffmaps import compute_difference_map, max_negentropy_kweighted_difference_map
from meteor.rsmap import Map
from meteor.tv import tv_denoise_difference_map
from meteor.validate import map_negentropy

In [2]:
# path is relative to where this notebook usually is in the `meteor` repo
mtz_path = "../meteor/test/data/scaled-test-data.mtz"
mtz_dataset = rs.read_mtz(mtz_path)

print(mtz_dataset.columns)
mtz_dataset

Index(['F_on', 'SIGF_on', 'F_off', 'SIGF_off', 'FC_nochrom', 'PHIC_nochrom',
       'F_on_scaled', 'SIGF_on_scaled', 'F_off_scaled', 'SIGF_off_scaled',
       'F_k', 'SIGF_k', 'PHI_k', 'F_TV', 'PHI_TV', 'SIGF_TV', 'F_itTV',
       'SIGF_itTV', 'PHI_itTV'],
      dtype='object')


F_on   SIGF_on      F_off  SIGF_off  FC_nochrom  PHIC_nochrom  \
H  K  L                                                                        
0  0  4    8.011057  0.863579   9.283087  0.958057  240.917175    180.000015   
      6   30.975687  2.312218  25.823608  2.077911  502.529755    180.000015   
      8   18.505907  2.773533  15.957798  2.617453  524.541382    180.000015   
      10  38.632221  3.155106  29.471071  2.552977   439.58493         360.0   
      12  21.541544  1.861844  22.385498   2.05849  244.351562    180.000015   
...             ...       ...        ...       ...         ...           ...   
34 11 6    1.802522   0.96382   1.840262  0.896258     6.59658      7.132044   
   12 2    1.040703  0.618049   1.062676   0.64162   10.618246    188.407745   
      5    1.585076  0.736849   2.405357   0.94527   12.357416    305.263458   
35 2  0    1.318665  0.889595   7.111948  0.861651   15.280899     90.000008   
      4    1.023447  0.704058   3.919934  1.295623   18.458782     92.787331   

          F_on_scaled  SIGF_on_scaled  F_off_scaled  SIGF_off_scaled  \
H  K  L                                                                
0  0  4    111.721573       12.043403    135.138657        13.946932   
      6    428.997192       32.023022    373.048096        30.017523   
      8    253.819321       38.040627    228.058289        37.406898   
      10   523.286194       42.736958    415.391968        35.983963   
      12   287.366577       24.837204    310.229065        28.527554   
...               ...             ...           ...              ...   
34 11 6      5.810488        3.106906      6.047815         2.945451   
   12 2      3.347993        1.988295      3.460584         2.089425   
      5      5.068055         2.35597      7.827047         3.075915   
35 2  0      4.120879        2.780018     22.406181         2.714631   
      4      3.193046        2.196586     12.406024          4.10046   

                F_k    SIGF_k       PHI_k      F_TV      PHI_TV   SIGF_TV  \
H  K  L                                                                     
0  0  4     4.29962  3.383419    0.000019  4.104198  359.827698  3.383419   
      6    2.132432  1.672898 -179.999985  1.889815  178.712463  1.672898   
      8     0.71406  1.478825 -179.999985  0.519908  186.646011  1.478825   
      10   2.368493  1.226426     0.00001  1.419956  359.240112  1.226426   
      12     1.2202  2.018752    0.000019  0.446494    3.752057  2.018752   
...             ...       ...         ...       ...         ...       ...   
34 11 6    0.193971  3.499082  -172.86795  0.021647  193.223831  3.499082   
   12 2    0.102222  2.618628    8.407731  0.010754   56.616051  2.618628   
      5    2.323771  3.263322  125.263443  0.130234   59.822773  3.263322   
35 2  0   13.340703  2.834864  -89.999992  1.054792   283.01886  2.834864   
      4    7.026342  3.547688  -87.212669  0.735626  268.623535  3.547688   

            F_itTV  SIGF_itTV    PHI_itTV  
H  K  L                                    
0  0  4   4.263433   3.368434  345.693268  
      6   1.914395   1.671889  185.970078  
      8   0.537505   1.478361  201.736832  
      10  1.464422   1.226652  355.365723  
      12  0.430938    2.01839  350.278625  
...            ...        ...         ...  
34 11 6   0.031937   3.499079  216.712357  
   12 2     0.0155   2.618614  352.686859  
      5   0.132257   3.263343   46.157078  
35 2  0   1.069749   2.835721  282.667603  
      4   0.776384   3.547976  267.622711  

[41429 rows x 19 columns]

In [ ]:
mtz_path = "../data/meteor_data/k.mtz"


# Test Column Types

In [3]:
mtz_path = "../meteor/test/data/scaled-test-data.mtz"
mtz_path = "../data/meteor_data/k.mtz"
mtz_path2= "../synthetic_cistrans/trans_sf.mtz"
mtzg= gemmi.read_mtz_file(mtz_path)
mtzg2= gemmi.read_mtz_file(mtz_path2)

print( mtzg.column_with_label("F_k"))
print( mtzg.column_with_label("PHI_k"))

print( mtzg2.column_with_label("FC"))
print( mtzg2.column_with_label("PHIC"))


<gemmi.Mtz.Column F_k type F>
<gemmi.Mtz.Column PHI_k type P>
<gemmi.Mtz.Column FC type F>
<gemmi.Mtz.Column PHIC type P>


# Rename Columns

In [4]:
mtz_path2= "../synthetic_cistrans/trans_sf.mtz"
new = "../synthetic_cistrans/trans_sf2.mtz"
mtzr2= rs.read_mtz(mtz_path2)
mtzr2["F"] = rs.DataSeries(mtzr2["FC"], dtype="F")
mtzr2["SIGF"] = np.ones(len(mtzr2["FC"]))
mtzr2["PHI"] = rs.DataSeries(mtzr2["PHIC"], dtype="P")
mtzr2["SIGF"] = rs.DataSeries(mtzr2["SIGF"], dtype="Q")
rs.utils.add_rfree(mtzr2,inplace=True)
del(mtzr2["FC"])
del(mtzr2["PHIC"])
mtzr2.write_mtz(new)
mtzr2

F  SIGF         PHI  R-free-flags
H  K L                                              
0  0 2   5083.867676   1.0         0.0             0
     4    218.642731   1.0  179.999985             0
     6    738.122253   1.0  179.999985             0
     8    746.586731   1.0  180.000015             0
     10   290.741333   1.0    0.000002             0
...              ...   ...         ...           ...
32 5 5      22.26017   1.0  334.031342             0
   6 0      8.205142   1.0  180.000229             0
     1     44.349613   1.0   98.977188             0
     2     32.996357   1.0  198.998627             0
     3     41.629765   1.0  200.730804             0

[31852 rows x 4 columns]

In [5]:
mtz_path2= "../synthetic_cistrans/100ps_sf.mtz"
new =      "../synthetic_cistrans/100ps_sf2.mtz"
mtzr2= rs.read_mtz(mtz_path2)
mtzr2["F"] = mtzr2["FC"]
mtzr2["PHI"] = mtzr2["PHIC"]
mtzr2["SIGF"] = np.ones(len(mtzr2["FC"]))
mtzr2["SIGF"] = rs.DataSeries(mtzr2["SIGF"], dtype="Q")
# mtzr2.infer_mtz_dtypes(inplace=True)
del(mtzr2["FC"])
del(mtzr2["PHIC"])
rs.utils.add_rfree(mtzr2,inplace=True)
mtzr2.write_mtz(new)
mtzr2



F         PHI  SIGF  R-free-flags
H  K L                                              
0  0 2   5099.616211         0.0   1.0             0
     4    180.751282  179.999985   1.0             1
     6    709.383118  180.000015   1.0             0
     8    752.830994  180.000015   1.0             0
     10   263.784851    0.000003   1.0             1
...              ...         ...   ...           ...
32 5 5     23.879597  330.984344   1.0             0
   6 0      8.442408  180.000259   1.0             0
     1     46.126518   97.791443   1.0             0
     2     32.960667  197.921204   1.0             0
     3       40.5933  200.852386   1.0             0

[31852 rows x 4 columns]

In [10]:
mtzg2.column_labels()
mtz_path2= "../synthetic_cistrans/trans_sf.mtz"

['H', 'K', 'L', 'FC', 'PHIC']

# Overwrite Columns from other dataset

In [11]:
mtzr= rs.read_mtz(mtz_path)
mtzr2= rs.read_mtz(mtz_path2)
mtzr["F_alt"] = mtzr2["FC"]
mtzr["PHI_alt"] = mtzr2["PHIC"]
mtzr.write_mtz("../synthetic_cistrans/trans_sf3.mtz")

In [32]:
mtz_path3 = "../data/meteor_data/on.mtz"
mtz_path4= "../synthetic_cistrans/100ps_sf.mtz"
mtz_new = "../synthetic_cistrans/100ps_sf3.mtz"
mtzr3= rs.read_mtz(mtz_path3)
print(mtzr3.columns)
mtzr4= rs.read_mtz(mtz_path4)
mtzr3["F_on"] = mtzr4["FC"]
if False:
    mtzr3["SIGF_on"] = mtzr3["SIGF_on"]-mtzr3["SIGF_on"]+0.1


idx = mtzr3.index[-1]
na = mtzr3.loc[idx, "F_on"]
print("na: ", na)
keep_or_nan = [elem is not na for elem in mtzr3["F_on"]]
mtzr3=mtzr3[keep_or_nan]
mtzr3.write_mtz(mtz_new)
mtzr3

Index(['F_on', 'SIGF_on', 'FreeR_flag'], dtype='object')
na:  <NA>


F_on   SIGF_on  FreeR_flag
H  K L                                   
0  0 4   180.751282  0.863579           0
     6   709.383118  2.312218           0
     8   752.830994  2.773533           0
     10  263.784851  3.155106           0
     12  248.778351  1.861844           0
...             ...       ...         ...
32 5 5    23.879597  1.185403           0
   6 0     8.442408  1.172047           0
     1    46.126518  0.857863           0
     2    32.960667  0.815381           0
     3      40.5933   0.76498           0

[31837 rows x 3 columns]

In [31]:
mtz_path3 = "../data/meteor_data/k.mtz"
mtz_path4= "../synthetic_cistrans/trans_sf.mtz"
mtz_new = "../synthetic_cistrans/trans_sf3.mtz"
mtzr3= rs.read_mtz(mtz_path3)
print(mtzr3.columns)
mtzr4= rs.read_mtz(mtz_path4)
mtzr3["F_k"] = mtzr4["FC"]
mtzr3["PHI_k"] = mtzr4["PHIC"]
if False:
    mtzr3["SIGF_k"] = mtzr3["SIGF_k"]-mtzr3["SIGF_k"]+0.1

idx = mtzr3.index[-1]
na = mtzr3.loc[idx, "F_k"]
print("na: ", na)
keep_or_nan = [elem is not na for elem in mtzr3["F_k"]]
mtzr3=mtzr3[keep_or_nan]
mtzr3.write_mtz(mtz_new)
mtzr3

Index(['F_k', 'SIGF_k', 'PHI_k', 'FreeR_flag'], dtype='object')
na:  <NA>


F_k    SIGF_k       PHI_k  FreeR_flag
H  K L                                               
0  0 4   218.642731  3.383419  179.999985           0
     6   738.122253  1.672898  179.999985           0
     8   746.586731  1.478825  180.000015           0
     10  290.741333  1.226426    0.000002           0
     12  234.383102  2.018752  180.000015           0
...             ...       ...         ...         ...
32 5 5     22.26017  3.830057  334.031342           0
   6 0     8.205142  3.998422  180.000229           0
     1    44.349613  3.266269   98.977188           0
     2    32.996357  3.488757  198.998627           0
     3    41.629765  3.368649  200.730804           0

[31837 rows x 4 columns]

In [29]:
mtz_path3 = "../data/meteor_data/on.mtz"

mtz_path4 = "../synthetic_cistrans/100ps_withSIGFC_amplitudenoise.mtz"
mtz_new  =  "../synthetic_cistrans/100ps_noise2.mtz"
mtzr3= rs.read_mtz(mtz_path3)
mtzr4= rs.read_mtz(mtz_path4)
print(mtzr3.columns)
print(mtzr4.columns)
mtzr3["F_on"] = mtzr4["FC"]
mtzr3["SIGF_on"] = mtzr4["SIGFC"]

idx = mtzr3.index[-1]
na = mtzr3.loc[idx, "F_on"]
print("na: ", na)
keep_or_nan = [elem is not na for elem in mtzr3["F_on"]]
mtzr3=mtzr3[keep_or_nan]
mtzr3.write_mtz(mtz_new)
mtzr3

Index(['F_on', 'SIGF_on', 'FreeR_flag'], dtype='object')
Index(['FC', 'PHIC', 'SIGFC'], dtype='object')
na:  <NA>


F_on  SIGF_on  FreeR_flag
H  K L                                  
0  0 4   174.947159      1.0           0
     6   705.675903      1.0           0
     8   782.494568      1.0           0
     10  280.375092      1.0           0
     12  257.441864      1.0           0
...             ...      ...         ...
32 5 5    24.470146      1.0           0
   6 0     8.271152      1.0           0
     1     46.57737      1.0           0
     2    33.660633      1.0           0
     3     42.74498      1.0           0

[31837 rows x 3 columns]

In [30]:
mtz_path3 = "../data/meteor_data/k.mtz"
mtz_path4= "../synthetic_cistrans/trans_withSIGFC_amplitudenoise.mtz"
mtz_new = "../synthetic_cistrans/trans_noise2.mtz"

mtzr3= rs.read_mtz(mtz_path3)
print(mtzr3.columns)
print(mtzr4.columns)
mtzr4= rs.read_mtz(mtz_path4)
mtzr3["F_k"] = mtzr4["FC"]
mtzr3["PHI_k"] = mtzr4["PHIC"]
mtzr3["SIGF_k"] = mtzr4["SIGFC"]

idx = mtzr3.index[-1]
na = mtzr3.loc[idx, "F_k"]
print("na: ", na)
keep_or_nan = [elem is not na for elem in mtzr3["F_k"]]
mtzr3=mtzr3[keep_or_nan]
mtzr3.write_mtz(mtz_new)
mtzr3

Index(['F_k', 'SIGF_k', 'PHI_k', 'FreeR_flag'], dtype='object')
Index(['FC', 'PHIC', 'SIGFC'], dtype='object')
na:  <NA>


F_k  SIGF_k       PHI_k  FreeR_flag
H  K L                                             
0  0 4   207.496155     1.0  179.999985           0
     6   701.769043     1.0  179.999985           0
     8   762.763062     1.0  180.000015           0
     10  305.082855     1.0    0.000002           0
     12   232.17955     1.0  180.000015           0
...             ...     ...         ...         ...
32 5 5    23.653811     1.0  334.031342           0
   6 0     8.001189     1.0  180.000229           0
     1    47.380619     1.0   98.977188           0
     2    31.159575     1.0  198.998627           0
     3    41.228428     1.0  200.730804           0

[31837 rows x 4 columns]

# a little bit of noise

In [ ]:
name_true= "../synthetic_cistrans/trans_sf.mtz"
name_old = "../synthetic_cistrans/trans_noise2.mtz"
name_new = "../synthetic_cistrans/trans_noise3.mtz"
noise_level = 0.01  

mtzmaster = rs.read_mtz(name_old)
mtztrue = rs.read_mtz(name_true)

mtzmaster["SIGFC"] = 0.2
mtzmaster["SIGFC"] = mtzmaster["SIGFC"].astype("Q")

fcol= "F_k"
original_fc = mtztrue["FC"].to_numpy()
noise = np.random.normal(loc=0.0, scale=noise_level * np.abs(original_fc))
mtzmaster[fcol] = mtztrue["FC"] + noise
mtzmaster.write_mtz(name_new)

In [44]:
name_true= "../synthetic_cistrans/100ps_sf.mtz"
name_old = "../synthetic_cistrans/100ps_noise2.mtz"
name_new = "../synthetic_cistrans/100ps_noise3.mtz"
mtzmaster = rs.read_mtz(name_old)
mtztrue = rs.read_mtz(name_true)

fcol= "F_on"
noise_level = 0.01  

mtzmaster["SIGFC"] = 0.2
mtzmaster["SIGFC"] = mtzmaster["SIGFC"].astype("Q")

original_fc = mtztrue["FC"].to_numpy()
noise = np.random.normal(loc=0.0, scale=noise_level * np.abs(original_fc))
mtzmaster[fcol] = mtztrue["FC"] + noise
mtzmaster.write_mtz(name_new)


0_1


# a lot of of noise

In [4]:
name_true= "../synthetic_cistrans/100ps_sf.mtz"
name_old = "../synthetic_cistrans/100ps_noise2.mtz"
name_new = "../synthetic_cistrans/100ps_noise4.mtz"
mtzmaster = rs.read_mtz(name_old)
mtztrue = rs.read_mtz(name_true)

fcol= "F_on"
noise_level = 0.2  

mtzmaster["SIGFC"] = 5
mtzmaster["SIGFC"] = mtzmaster["SIGFC"].astype("Q")

original_fc = mtztrue["FC"].to_numpy()
noise = np.random.normal(loc=0.0, scale=noise_level * np.abs(original_fc))
mtzmaster[fcol] = mtztrue["FC"] + noise
mtzmaster.write_mtz(name_new)


In [5]:
name_true= "../synthetic_cistrans/trans_sf.mtz"
name_old = "../synthetic_cistrans/trans_noise2.mtz"
name_new = "../synthetic_cistrans/trans_noise4.mtz"
noise_level = 0.2  

mtzmaster = rs.read_mtz(name_old)
mtztrue = rs.read_mtz(name_true)

mtzmaster["SIGFC"] = 5
mtzmaster["SIGFC"] = mtzmaster["SIGFC"].astype("Q")

fcol= "F_k"
original_fc = mtztrue["FC"].to_numpy()
noise = np.random.normal(loc=0.0, scale=noise_level * np.abs(original_fc))
mtzmaster[fcol] = mtztrue["FC"] + noise
mtzmaster.write_mtz(name_new)


In [ ]:
%gemmi sfcalc -v --to-mtz=trans_sf.mtz --dmin 1.6 trans.pdb

In [15]:
dataloc = "../synthetic_cistrans/"
pdbname_light = dataloc+"100ps.pdb"
folder_location = "../meteor/test/data/"
pdbname_light = folder_location+"8a6g-chromophore-removed.pdb"
struc_light = gemmi.read_structure(pdbname_light)
pdbname_dark = folder_location+"8a6g-chromophore-removed.pdb"
struc_dark = gemmi.read_structure(pdbname_dark)


In [21]:
print(pdbname_light)
pdbname_light
def pdb2mtz(pdbname):
    conv_mtz = pdbname[:-3] + "mtz"
    !echo $pdbname $conv_mtz
    !gemmi sfcalc -v --to-mtz=$conv_mtz --dmin 1.6 $pdbname
pdb2mtz(pdbname_light)

../meteor/test/data/8a6g-chromophore-removed.pdb
../meteor/test/data/8a6g-chromophore-removed.pdb ../meteor/test/data/8a6g-chromophore-removed.mtz
Reading file ../meteor/test/data/8a6g-chromophore-removed.pdb ...
B_min=14.15, B_add=6.26712
Preparing electron density on a grid...
...took 0.217528 s.
FFT of grid 100 x 120 x 144
...took 0.0100322 s.
Printing results...
